Demo

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/task2/Question-Generator/
!ls

/content/drive/MyDrive/task2/Question-Generator
 Data			  input1.txt	  'Question Generation'
'Final Demo.ipynb'	  input_file.txt
 generatedQuestions.txt   out_text.txt


In [ ]:
import nltk
import warnings
warnings.filterwarnings('ignore')
import spacy
from spacy import displacy
import pandas as pd
import random

In [ ]:
 def startgeneration(input_file):
    print ('\nChoose from the following : \n1. Fill in the Blanks \n2. Multiple Choice Questions \n3. True or False Questions')

    file = open (input_file , 'r')
    lines = file.readlines()
    text = ''

    for i in lines:
        text = text + i

    text = ' '.join(text.split("\n"))
    print(len(text))
    option = int (input())
    
    flag = 0
    
    if (option == 1):
        print ('\nFill in the Blanks Question Generation Starting...')
        flag = 1
    elif (option == 2):
        print ('\nMultiple Choice Question Generation Starting...')   
        flag = 1
    elif (option == 3):
        print ('\nTrue or False Question Generation Starting...')
        flag = 1
    else:
        print ('\nInvalid Input')
        flag = 0
    
    if (flag == 1):
        generateQuestions(text, option)
        print ('Questions Generated and saved in : generatedQuestions.txt')

Pickling

In [ ]:
!pip install pickle5
import pickle5 as cPickle
from pathlib import Path

def dumpPickle(fileName, content):
    pickleFile = open(fileName, 'wb')
    cPickle.dump(content, pickleFile, -1)
    pickleFile.close()

def loadPickle(fileName):    
    file = open(fileName, 'rb')
    content = cPickle.load(file)
    file.close()
    
    return content
    
def pickleExists(fileName):
    file = Path(fileName)
    
    if file.is_file():
        return True
    
    return False

     |████████████████████████████████| 133kB 5.6MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219268 sha256=486b403469f49ed28d8c32ab66a5ace6070a75a599a67cabf66baf58055f2413
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


Extract all words from plain text and generate it's features

In [ ]:
nlp = spacy.load('en_core_web_sm')

#Extract answers and the sentence they are in
def extractAnswers(qas, doc):
    answers = []

    senStart = 0
    senId = 0

    for sentence in doc.sents:
        senLen = len(sentence.text)

        for answer in qas:
            answerStart = answer['answers'][0]['answer_start']

            if (answerStart >= senStart and answerStart < (senStart + senLen)):
                answers.append({'sentenceId': senId, 'text': answer['answers'][0]['text']})

        senStart += senLen
        senId += 1
    
    return answers

#TODO - Clean answers from stopwords?
def tokenIsAnswer(token, sentenceId, answers):
    for i in range(len(answers)):
        if (answers[i]['sentenceId'] == sentenceId):
            if (answers[i]['text'] == token):
                return True
    return False

#Save named entities start points

def getNEStartIndexs(doc):
    neStarts = {}
    for ne in doc.ents:
        neStarts[ne.start] = ne
        
    return neStarts 

def getSentenceStartIndexes(doc):
    senStarts = []
    
    for sentence in doc.sents:
        senStarts.append(sentence[0].i)
    
    return senStarts
    
def getSentenceForWordPosition(wordPos, senStarts):
    for i in range(1, len(senStarts)):
        if (wordPos < senStarts[i]):
            return i - 1
        
def addWordsForParagrapgh(newWords, text):
    doc = nlp(text)

    neStarts = getNEStartIndexs(doc)
    senStarts = getSentenceStartIndexes(doc)
    
    #index of word in spacy doc text
    i = 0
    
    while (i < len(doc)):
        #If the token is a start of a Named Entity, add it and push to index to end of the NE
        if (i in neStarts):
            word = neStarts[i]
            #add word
            currentSentence = getSentenceForWordPosition(word.start, senStarts)
            wordLen = word.end - word.start
            shape = ''
            for wordIndex in range(word.start, word.end):
                shape += (' ' + doc[wordIndex].shape_)

            newWords.append([word.text,
                            0,
                            0,
                            currentSentence,
                            wordLen,
                            word.label_,
                            None,
                            None,
                            None,
                            shape])
            i = neStarts[i].end - 1
        #If not a NE, add the word if it's not a stopword or a non-alpha (not regular letters)
        else:
            if (doc[i].is_stop == False and doc[i].is_alpha == True):
                word = doc[i]

                currentSentence = getSentenceForWordPosition(i, senStarts)
                wordLen = 1

                newWords.append([word.text,
                                0,
                                0,
                                currentSentence,
                                wordLen,
                                None,
                                word.pos_,
                                word.tag_,
                                word.dep_,
                                word.shape_])
        i += 1

def oneHotEncodeColumns(df):
    columnsToEncode = ['NER', 'POS', "TAG", 'DEP']

    for column in columnsToEncode:
        one_hot = pd.get_dummies(df[column])
        one_hot = one_hot.add_prefix(column + '_')

        df = df.drop(column, axis = 1)
        df = df.join(one_hot)
    
    return df

Predict whether a word is a keyword

In [ ]:
def generateDf(text):
    words = []
    addWordsForParagrapgh(words, text)

    wordColums = ['text', 'titleId', 'paragrapghId', 'sentenceId','wordCount', 'NER', 'POS', 'TAG', 'DEP','shape']
    df = pd.DataFrame(words, columns=wordColums)
    
    return df

In [ ]:
def prepareDf(df):
    #One-hot encoding
    wordsDf = oneHotEncodeColumns(df)

    #Drop unused columns
    columnsToDrop = ['text', 'titleId', 'paragrapghId', 'sentenceId', 'shape']
    wordsDf = wordsDf.drop(columnsToDrop, axis = 1)

    #Add missing colums 
    predictorColumns = ['wordCount', 'NER_CARDINAL', 'NER_DATE', 'NER_EVENT', 'NER_FAC',
       'NER_GPE', 'NER_LANGUAGE', 'NER_LAW', 'NER_LOC', 'NER_MONEY',
       'NER_NORP', 'NER_ORDINAL', 'NER_ORG', 'NER_PERCENT', 'NER_PERSON',
       'NER_PRODUCT', 'NER_QUANTITY', 'NER_TIME', 'NER_WORK_OF_ART', 'POS_ADJ',
       'POS_ADP', 'POS_ADV', 'POS_CCONJ', 'POS_DET', 'POS_NOUN', 'POS_NUM',
       'POS_PROPN', 'POS_SCONJ', 'POS_VERB', 'TAG_CC', 'TAG_CD', 'TAG_IN',
       'TAG_JJ', 'TAG_JJR', 'TAG_JJS', 'TAG_NN', 'TAG_NNP', 'TAG_NNPS',
       'TAG_NNS', 'TAG_PDT', 'TAG_RB', 'TAG_RBR', 'TAG_RBS', 'TAG_VB',
       'TAG_VBD', 'TAG_VBG', 'TAG_VBN', 'TAG_VBP', 'TAG_VBZ', 'DEP_ROOT',
       'DEP_acl', 'DEP_acomp', 'DEP_advcl', 'DEP_advmod', 'DEP_agent',
       'DEP_amod', 'DEP_appos', 'DEP_attr', 'DEP_aux', 'DEP_auxpass',
       'DEP_ccomp', 'DEP_compound', 'DEP_conj', 'DEP_csubj', 'DEP_csubjpass',
       'DEP_dative', 'DEP_dep', 'DEP_dobj', 'DEP_nmod', 'DEP_npadvmod',
       'DEP_nsubj', 'DEP_nsubjpass', 'DEP_nummod', 'DEP_oprd', 'DEP_pcomp',
       'DEP_pobj', 'DEP_poss', 'DEP_predet', 'DEP_prep', 'DEP_relcl',
       'DEP_xcomp']
    for feature in predictorColumns:
        if feature not in wordsDf.columns:
            wordsDf[feature] = 0
    return wordsDf

In [ ]:
def predictWords(wordsDf, df):
    
    predictorPickleName = 'Data/nb-predictor.pkl'
    predictor = loadPickle(predictorPickleName)
    
    y_pred = predictor.predict_proba(wordsDf)

    labeledAnswers = []
    for i in range(len(y_pred)):
        labeledAnswers.append({'word': df.iloc[i]['text'], 'prob': y_pred[i][0]})
    
    return labeledAnswers

Extract questions

In [ ]:
def blankAnswer(firstTokenIndex, lastTokenIndex, sentStart, sentEnd, doc):
    leftPartStart = doc[sentStart].idx
    leftPartEnd = doc[firstTokenIndex].idx
    rightPartStart = doc[lastTokenIndex].idx + len(doc[lastTokenIndex])
    rightPartEnd = doc[sentEnd - 1].idx + len(doc[sentEnd - 1])
    
    question = doc.text[leftPartStart:leftPartEnd] + '_____' + doc.text[rightPartStart:rightPartEnd]
    
    return question

In [ ]:
def addQuestions(answers, text):
    doc = nlp(text)
    currAnswerIndex = 0
    qaPair = []

    #Check wheter each token is the next answer
    for sent in doc.sents:
        for token in sent:
            
            #If all the answers have been found, stop looking
            if currAnswerIndex >= len(answers):
                break
            
            #In the case where the answer is consisted of more than one token, check the following tokens as well.
            answerDoc = nlp(answers[currAnswerIndex]['word'])
            answerIsFound = True
            
            for j in range(len(answerDoc)):
                if token.i + j >= len(doc) or doc[token.i + j].text != answerDoc[j].text:
                    answerIsFound = False
           
            #If the current token is corresponding with the answer, add it 
            if answerIsFound:
                question = blankAnswer(token.i, token.i + len(answerDoc) - 1, sent.start, sent.end, doc)
                
                qaPair.append({'question' : question, 'answer': answers[currAnswerIndex]['word'], 'prob': answers[currAnswerIndex]['prob']})
                
                currAnswerIndex += 1
                
    return qaPair

In [ ]:
def sortAnswers(qaPairs):
    orderedQaPairs = sorted(qaPairs, key=lambda qaPair: qaPair['prob'])
    
    return orderedQaPairs    

Distractors (Incorrect Answers)

In [ ]:
path = 'Data/distractor.pkl'
model = loadPickle(path)

In [ ]:
def generate_distractors(answer, count):
    answer = str.lower(answer)
    
    ##Extracting closest words for the answer. 
    try:
        closestWords = model.most_similar(positive=[answer], topn=count)
    except:
        return []
    
    distractors = list(map(lambda x: x[0], closestWords))[0:count]
    
    return distractors

In [ ]:
def addDistractors(qaPairs, count):
    for qaPair in qaPairs:
        distractors = generate_distractors(qaPair['answer'], count)
        qaPair['distractors'] = distractors
    
    return qaPairs

Question Generation Function

In [ ]:
def generateQuestions(text, option):
    
    # Extract words 
    df = generateDf(text)
    wordsDf = prepareDf(df)

    # Predict 
    labeledAnswers = predictWords(wordsDf, df)

    
    # Transform questions
    qaPairs = addQuestions(labeledAnswers, text)
    
    # Pick the best questions
    orderedQaPairs = sortAnswers(qaPairs)
    
    # Generate distractors
    questions = addDistractors(orderedQaPairs[:len(orderedQaPairs)], 3)

    # Formating the output into a txt file
    path = 'generatedQuestions.txt'
    file = open(path, 'w')
    
    if (option == 1):
        for i in range(len(orderedQaPairs)): 
            temp = 'Question ' + str(i+1) + '. '
            file.write(temp)
            file.write(questions[i]['question'])
            file.write('\n')
            file.write('Answer : ')
            file.write(questions[i]['answer'])
            file.write('\n\n')     
    
    if (option == 2):
        choice = ['a. ', 'b. ', 'c. ', 'd. ']
        for i in range(len(orderedQaPairs)):
            temp = 'Question ' + str(i+1) + '. ' 
            file.write(temp)
            file.write(questions[i]['question'])
            file.write('\n')
            options = []
            a = (questions[i]['answer']).lower()
            options.append(a)

            for distractor in questions[i]['distractors']:
                options.append(distractor)
                
            random.shuffle(options)
            for j in range (len(options)):
                temp2 = choice[j] + options[j]
                file.write(temp2)
                file.write('\n') 
            temp3 = 'Answer : ' + a
            file.write(temp3)
            file.write('\n\n')
            
    if (option == 3):
        for i in range(len(orderedQaPairs)):
            temp = 'Question ' + str(i+1) + '. ' 
            file.write(temp)
            options = []
            options.append(questions[i]['answer'])

            for distractor in questions[i]['distractors']:
                options.append(distractor)

            random.shuffle(options)
            inputOption = random.choice(options) 

            file.write(questions[i]['question'].replace('_____', inputOption))
            file.write('\n')
            file.write('Answer : ')

            if (inputOption == questions[i]['answer']):
                file.write ('True')

            else:
                file.write ('False')
            
            file.write ('\n\n')

In [30]:
input_file = "input1.txt"
#input paragraph = "" #@param {type:"string"}
print ('\nSpell check done and changes made and saved in : ', input_file)
startgeneration(input_file)


Spell check done and changes made and saved in :  input1.txt

Choose from the following : 
1. Fill in the Blanks 
2. Multiple Choice Questions 
3. True or False Questions
130
2

Multiple Choice Question Generation Starting...
Questions Generated and saved in : generatedQuestions.txt
